In [1]:
import requests
import pandas as pd
import pymysql
import numpy as np
from datetime import datetime

stable_coins = "USDT, USDC, DAI, BUSD, SUSD, UST, TUSD".split(",")

In [2]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

def insert_raydium_data_df(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        dt = row["Date"].timestamp()
        try:
            pair2 = row["Pair_2"] if row["Pair_2"] else ' '
            print(row["Date"], row["Pair_1"], pair2, row["Pool_rate"], row["Farm_payment"], row["Where"], row["includes_stablecoin"])
            sql = "INSERT INTO `raydium_farm_data` (`timestamp_dt`, `pair1`,`pair2`,`pool_rate`, `farm_payment`,`where`,`includes_stablecoin`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (dt, row["Pair_1"], pair2, row["Pool_rate"], row["Farm_payment"], row["Where"], row["includes_stablecoin"]))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()

#### https://raydium.io/pools/

In [6]:
raydium_site_pairs = 'https://api.raydium.io/pairs'
pair_data = requests.get(raydium_site_pairs)
pair_data = pair_data.json()

In [7]:
pair_data[0]

{'name': 'SBFC-USDC',
 'pair_id': 'AWW5UQfMBnPsTaaxCK7cSEmkj1kbX2zUrqvgKXStjBKx-EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
 'lp_mint': '5XiU5T79Y4AzCK7ZBZLrRMApjPNvKdhn1zP7TS2VE6Gk',
 'official': False,
 'liquidity': 298.589856,
 'market': '3ivg8YYCsSnijDDKFgbrWMZJc377nGH8TQnNNbdEaDdJ',
 'volume_24h': 86.23266199999995,
 'volume_24h_quote': 86.23266199999995,
 'fee_24h': 0.21558165499999987,
 'fee_24h_quote': 0.21558165499999987,
 'volume_7d': 538.613222,
 'volume_7d_quote': 538.613222,
 'fee_7d': 1.3465330549999999,
 'fee_7d_quote': 1.3465330549999999,
 'price': 0.0004236729298834144,
 'amm_id': '14MDXFwbr5gPdzTS43XBD6oZEjiKifTziujR1216jrBe',
 'apy': 26.35}

In [8]:
pairs = []
for data in pair_data:
    pairs.append([data['name'], data['liquidity'], data['fee_24h'], data['liquidity'], data['price'], data['apy'], data['volume_7d']])

In [9]:
raydiom_pool_df = pd.DataFrame(pairs, columns=['name', 'tvl', 'fee24h', 'liquidity', 'price', 'Pool_rate', 'volume_7d'])

In [10]:
raydiom_pool_df[['Pair_1', 'Pair_2']] = raydiom_pool_df['name'].str.split("-", expand=True)[[0,1]]
raydiom_pool_df['Farm_payment'] = np.nan
raydiom_pool_df['Where'] = 'Raydium'
raydiom_pool_df['includes_stablecoin'] = (raydiom_pool_df['Pair_1'].isin(stable_coins)) | (raydiom_pool_df['Pair_2'].isin(stable_coins))

now = datetime.now()
now = datetime(now.year, now.month, now.day)
raydiom_pool_df['Date'] = now


In [11]:
raydiom_pool_df

,name,tvl,fee24h,liquidity,price,Pool_rate,volume_7d,Pair_1,Pair_2,Farm_payment,Where,includes_stablecoin,Date
0,SBFC-USDC,2.985899e+02,0.215582,2.985899e+02,0.000424,26.35,5.386132e+02,SBFC,USDC,NaN,Raydium,False,2021-06-16
1,FTT-SRM,1.144058e+05,8.163274,1.144058e+05,7.717012,2.60,1.673099e+04,FTT,SRM,NaN,Raydium,False,2021-06-16
2,KIN-USDC,5.339775e+01,0.006820,5.339775e+01,0.000053,4.66,4.802542e+01,KIN,USDC,NaN,Raydium,False,2021-06-16
3,FIDA-RAY,4.874651e+06,175.127005,4.874651e+06,0.508581,1.31,1.352929e+06,FIDA,RAY,NaN,Raydium,False,2021-06-16
4,FIDA-USDC,0.000000e+00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,FIDA,USDC,NaN,Raydium,False,2021-06-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,LINK-USDC,1.069012e+05,31.720085,1.069012e+05,23.709269,10.83,1.046737e+05,LINK,USDC,NaN,Raydium,False,2021-06-16
94,TYNA-SOL,1.151475e-01,0.000000,1.151475e-01,3000.000000,0.00,0.000000e+00,TYNA,SOL,NaN,Raydium,False,2021-06-16
95,SOL-oDOP,3.371325e+01,0.001187,3.371325e+01,2271.275472,1.28,3.786492e+01,SOL,oDOP,NaN,Raydium,False,2021-06-16
96,SOL-unknown,1.384052e+02,0.000000,1.384052e+02,6.695166,0.00,0.000000e+00,SOL,unknown,NaN,Raydium,False,2021-06-16


In [63]:
raydiom_pool_df.to_csv('raydium_data.csv')

In [66]:
insert_raydium_data_df(raydiom_pool_df)

2021-05-04 00:00:00 FTT USDC 43.91 None Raydium False
(1062, "Duplicate entry '2021-05-04 00:00:00-FTT-USDC' for key 'raydium_farm_data.PRIMARY'")
2021-05-04 00:00:00 RAY USDC 21.61 None Raydium False
(1062, "Duplicate entry '2021-05-04 00:00:00-RAY-USDC' for key 'raydium_farm_data.PRIMARY'")
2021-05-04 00:00:00 SOL USDT 55.92 None Raydium True
(1062, "Duplicate entry '2021-05-04 00:00:00-SOL-USDT' for key 'raydium_farm_data.PRIMARY'")
2021-05-04 00:00:00 COPE USDC 6.86 None Raydium False
(1062, "Duplicate entry '2021-05-04 00:00:00-COPE-USDC' for key 'raydium_farm_data.PRIMARY'")
2021-05-04 00:00:00 TOMO USDT 8.7 None Raydium True
(1062, "Duplicate entry '2021-05-04 00:00:00-TOMO-USDT' for key 'raydium_farm_data.PRIMARY'")
2021-05-04 00:00:00 RAY SRM 11.88 None Raydium False
(1062, "Duplicate entry '2021-05-04 00:00:00-RAY-SRM' for key 'raydium_farm_data.PRIMARY'")
2021-05-04 00:00:00 SUSHI USDC 31.96 None Raydium False
(1062, "Duplicate entry '2021-05-04 00:00:00-SUSHI-USDC' for key 